Running a scalability test with a random SNN.

In [1]:
import time
import random
from superneuroabm.model import NeuromorphicModel

In [2]:
def execute_run(graph, snn_config, n_inputs, spike_times, sim_time, use_gpu):

    t_init_start = time.time()
    model = NeuromorphicModel(use_cuda=use_gpu)
    t_init_end = time.time()
    t_init = t_init_end - t_init_start

    t_create_start = time.time()

    for _ in graph.nodes:
        _ = model.create_neuron(
            threshold=snn_config["threshold"],
            reset_state=snn_config["reset_val"],
            refactory_period=snn_config["refractory_period"],
        )
        # NOTE: Typo in superneuroabm.neuron class refactory -> refractory

    for edge in graph.edges:
        # wt_val = np.random.randint(1, 5)
        pre_neuron = edge[0]
        post_neuron = edge[1]
        model.create_synapse(
            pre_neuron_id=pre_neuron,
            post_neuron_id=post_neuron,
            weight=snn_config["syn_wt"],
            synaptic_delay=snn_config["syn_delay"],
        )

    # set the input neurons receiving external stimulus:
    spiking_neurons = random.choices(range(len(graph.nodes)), k=n_inputs)

    for n in spiking_neurons:
        for t in spike_times:
            model.spike(n, t, 1)

    model.setup(output_buffer_len=sim_time)

    t_create_end = time.time()
    t_create = t_create_end - t_create_start
    

    t_sim_start = time.time()
    # Simulate for sim_time steps:
    model.simulate(ticks=sim_time, update_data_ticks=sim_time)
    t_sim_end = time.time()
    t_sim = t_sim_end - t_sim_start
    
    t_mon_start = time.time()
    # Get spike times
    for neuron in range(len(graph.nodes)):
        net_spikes = model.get_spikes(neuron)

    t_mon_end = time.time()
    t_mon = t_mon_end - t_mon_start
    
    return t_init, t_create, t_sim, t_mon

In [3]:
# Loads the network onto the requested simulator and runs for the specified timesteps
# Simulation is run for 1000ms (1000 timesteps)

import numpy as np
import networkx as nx

# Define the simulation parameters:
connection_probability = 0.5
seed = 42
syn_delay = 1
syn_wt = 1.0
threshold = 1.0
refractory_period = 0
reset_val = 0
leak = 0.0
sim_time = 1000
spike_time_intervals = 10
spike_times = np.arange(1.0, sim_time, spike_time_intervals)
use_gpu = False

n_neurons = 100
n_inputs = int(0.3  * n_neurons)

# Create the networkx random graph with given connection probability:
np.random.seed(seed)
graph = nx.binomial_graph(n_neurons, connection_probability)

snn_config = {}
snn_config["threshold"] = threshold
snn_config["syn_wt"] = syn_wt
snn_config["syn_delay"] = syn_delay
snn_config["refractory_period"] = refractory_period
snn_config["reset_val"] = reset_val
snn_config["leak"] = leak
snn_config["n_neurons"] = n_neurons

# Run a sample simulation:
execute_run(
    graph, snn_config, n_inputs, spike_times, sim_time, use_gpu
)

Simulation Progress: 100%|████████████████████████████████████████████████████████| 1.00k/1.00k [00:31<00:00, 32.2Tick/s]


(4.649162292480469e-05,
 0.06945371627807617,
 31.402495861053467,
 0.01963949203491211)

In [4]:
import numpy as np
import pandas as pd

# Compare the scalability performance:
neurons = [100, 1000]
density = [0.25, 0.5]

tests = 3
seeds = [np.random.randint(0, 100000000) for i in range(tests)]

df = pd.DataFrame(columns = ['n_neurons', 'density', 'init_time', 'create_time', 'sim_time', 'mon_time', 'total_time'],
        index = [i for i in range(tests*len(neurons)*len(density))])

i=0
for t in range(tests):
    for n in neurons:
        for d in density:
            n_inputs = int(0.3*n)
            graph = nx.binomial_graph(n, d)
            t_init, t_create, t_sim, t_mon = execute_run(
                                                graph, snn_config, n_inputs, spike_times, sim_time, use_gpu
                                                )
            
            print(n, d, t_init, t_create, t_sim, t_mon)
            df.loc[i,'n_neurons'] = n
            df.loc[i,'density'] = d
            df.loc[i,'init_time'] = t_init
            df.loc[i,'create_time'] = t_create
            df.loc[i,'sim_time'] = t_sim
            df.loc[i,'mon_time'] = t_mon
            df.loc[i,'total_time'] = t_init + t_create + t_sim + t_mon
            i+=1
           

   n_neurons density init_time create_time sim_time mon_time total_time
0        NaN     NaN       NaN         NaN      NaN      NaN        NaN
1        NaN     NaN       NaN         NaN      NaN      NaN        NaN
2        NaN     NaN       NaN         NaN      NaN      NaN        NaN
3        NaN     NaN       NaN         NaN      NaN      NaN        NaN
4        NaN     NaN       NaN         NaN      NaN      NaN        NaN
5        NaN     NaN       NaN         NaN      NaN      NaN        NaN
6        NaN     NaN       NaN         NaN      NaN      NaN        NaN
7        NaN     NaN       NaN         NaN      NaN      NaN        NaN
8        NaN     NaN       NaN         NaN      NaN      NaN        NaN
9        NaN     NaN       NaN         NaN      NaN      NaN        NaN
10       NaN     NaN       NaN         NaN      NaN      NaN        NaN
11       NaN     NaN       NaN         NaN      NaN      NaN        NaN


Simulation Progress: 100%|████████████████████████████████████████████████████████| 1.00k/1.00k [00:25<00:00, 39.3Tick/s]


100 0.25 5.6743621826171875e-05 0.06421256065368652 25.82773447036743 0.019110918045043945


Simulation Progress: 100%|████████████████████████████████████████████████████████| 1.00k/1.00k [00:30<00:00, 33.0Tick/s]


100 0.5 5.6743621826171875e-05 0.06667661666870117 30.702418327331543 0.01955103874206543


Simulation Progress:  42%|████████████████████████▏                                 | 418/1.00k [06:06<08:30, 1.14Tick/s]Process ForkPoolWorker-16:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File 

KeyboardInterrupt: 

In [ ]:
display(df)